In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql.functions import regexp_extract
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
import json

In [3]:
spark = SparkSession \
        .builder \
        .master('local') \
        .appName('airflow log test') \
        .getOrCreate()
 # spark = SparkSession.builder.master('local').appName("airflow log test").getOrCreate() # change master

22/02/06 22:36:08 WARN Utils: Your hostname, Jungminui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.43 instead (on interface en0)
22/02/06 22:36:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/06 22:36:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
import pprint
pprint.pprint(spark.sparkContext.getConf().getAll())

[('spark.master', 'local'),
 ('spark.driver.port', '50861'),
 ('spark.sql.warehouse.dir',
  'file:/Users/mdgome/workspace/study/StudyDataEngineering/StudyDataEngineering/miniporject%232/code/spark-warehouse'),
 ('spark.app.name', 'airflow log test'),
 ('spark.app.startTime', '1644154569044'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', '172.30.1.43'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1644154571110'),
 ('spark.ui.showConsoleProgress', 'true')]


In [5]:
path = "/Users/mdgome/workspace/study/StudyDataEngineering/StudyDataEngineering/miniporject#2/code/MySQL_to_CSV.text"
# rdd = spark.sparkContext.textFile(path)
data = spark.read.text(path)

In [7]:
regex = r"\[((\d+-\d+-\d+) (\d+:\d+:\d+,\d+))\] \{\S+\} (DEBUG|INFO|WARN|FATAL|ERROR|TRACE) - ([a-zA-Z0-9]+.*)"

logs_df = data.select(regexp_extract('value', regex, 1).alias('datetime'),
                       regexp_extract('value', regex, 4).alias('log_level'),
                      regexp_extract('value', regex,5).alias('message')
                     )
logs_df.show(truncate=True)
print((logs_df.count(), len(logs_df.columns)))

+--------------------+---------+--------------------+
|            datetime|log_level|             message|
+--------------------+---------+--------------------+
|2022-01-19 11:46:...|     INFO|Started process (...|
|2022-01-19 11:46:...|     INFO|Processing file /...|
|2022-01-19 11:46:...|     INFO|Filling up the Da...|
|2022-01-19 11:46:...|     INFO|DAG(s) dict_keys(...|
|                    |         |                    |
|                    |         |                    |
|2022-01-19 11:46:...|     INFO|Created Permissio...|
|2022-01-19 11:46:...|     INFO|Created Permissio...|
|2022-01-19 11:46:...|     INFO|         Sync 1 DAGs|
|2022-01-19 11:46:...|     INFO|Creating ORM DAG ...|
|2022-01-19 11:46:...|     INFO|Setting next_dagr...|
|2022-01-19 11:46:...|     INFO|Processing /opt/a...|
|2022-01-19 11:46:...|     INFO|Started process (...|
|2022-01-19 11:46:...|     INFO|Processing file /...|
|2022-01-19 11:46:...|     INFO|Filling up the Da...|
|2022-01-19 11:47:...|     I

In [ ]:
logs_df = logs_df.withColumn('datetime',year

In [86]:
json_bulk = logs_df.toJSON().map(lambda x: json.loads(x)).collect()
json_bulk[0:5]

[{'datetime': '2022-01-19 11:46:20,775',
  'log_level': 'INFO',
  'message': 'Started process (PID=946) to work on /opt/airflow/dags/MySQL_to_CSV.py'},
 {'datetime': '2022-01-19 11:46:20,780',
  'log_level': 'INFO',
  'message': 'Processing file /opt/airflow/dags/MySQL_to_CSV.py for tasks to queue'},
 {'datetime': '2022-01-19 11:46:20,782',
  'log_level': 'INFO',
  'message': 'Filling up the DagBag from /opt/airflow/dags/MySQL_to_CSV.py'},
 {'datetime': '2022-01-19 11:46:22,634',
  'log_level': 'INFO',
  'message': "DAG(s) dict_keys(['second_assingnment']) retrieved from /opt/airflow/dags/MySQL_to_CSV.py"},
 {'datetime': '', 'log_level': '', 'message': ''}]

In [ ]:
type(json_bulk)
type(logs_df)

In [66]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es_client = Elasticsearch(hosts=['localhost:9200'],http_compress=True)
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'test_airflow_log',
                "_type": "_doc",
                "_id" : f"{document['id']}",
                "_source": filterKeys(document),
            } 
    raise StopIteration

use_these_keys = ['datetime', 'log_level', 'message']

def filterKeys(document):
    return {key: document[key] for key in use_these_keys }    


helpers.bulk(es_client, doc_generator(logs_df))

AttributeError: 'DataFrame' object has no attribute 'iterrows'

In [8]:
spark.stop()

In [67]:
!pwd

/Users/mdgome/workspace/study/StudyDataEngineering/StudyDataEngineering/miniporject#2/code
